
# Classifications of in-vivo Gastral Images - Training

## Imports
We import a set of Python standard libraries, as well as a range of Keras features.

In [26]:
import os
import re
import random
from shutil import copyfile

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

## Constants
We need to specify a set of "constants" (Python does not have constants, but it really *should* be constants ;-)) that depend on the data location of the machine that runs the code.
Note that you can get the data from https://rdm.inesctec.pt/dataset/nis-2018-003.

In [27]:
label_path = '/home/knapics1/CNN_LIME-master/trainingMAIN/labels.txt' # Path to labels file
data_path = '/home/knapics1/CNN_LIME-master/trainingMAIN/Set_1/A' # Path to data folder
working_path = '/home/knapics1/CNN_LIME-master/training' # Path for (to-be-prepared) training data

## Data Preparation
First, we need to prepare our data. We split data and labels into train and validation sets (randomly assigned) and, for each set, separate *bleeding* and *non-bleeding* images.

In [34]:
training_path = f'{working_path}/training'
validation_path = f'{working_path}/validation'
validation_split = 0.1

# read in labels
labels_file = open(label_path, 'r')
labels = labels_file.read().splitlines()

# get data set length
_, _, files = next(os.walk(data_path))
data_length = len(files)

print(f'Data length: {data_length}; label length: {len(labels)}')

# create directory structure
if not os.path.exists(training_path):
    os.makedirs(training_path)
if not os.path.exists(validation_path):
    os.makedirs(validation_path)

if not os.path.exists(f'{training_path}/1bleeding'):
    os.makedirs(f'{training_path}/1bleeding')
if not os.path.exists(f'{training_path}/0nonbleeding'):
    os.makedirs(f'{training_path}/0nonbleeding')
if not os.path.exists(f'{validation_path}/1bleeding'):
    os.makedirs(f'{validation_path}/1bleeding')
if not os.path.exists(f'{validation_path}/0nonbleeding'):
    os.makedirs(f'{validation_path}/0nonbleeding')

# copy files into directory structure
index = 0

# read in file in alphanumerical order
numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts
    
for _, _, files in os.walk(data_path):
    for file in sorted(files, key=numericalSort):
        base_path = training_path
        if random.uniform(0, 1) < validation_split: # assign pseudo-randomly to training or validation set
            base_path = validation_path
        if labels[index] == '0':
            copyfile(os.path.join(data_path, file), f'{base_path}/0nonbleeding/{file}') 
        else:
            copyfile(os.path.join(data_path, file), f'{base_path}/1bleeding/{file}')
        index = index + 1

Data length: 3295; label length: 3295
Training data length: 2993; validation data length: 302


In [ ]:
#Before next line copy the images you want to use to the newly generated folder training and validation

In [ ]:
_, _, files1 = next(os.walk(f'{training_path}/1bleeding'))
_, _, files2 = next(os.walk(f'{training_path}/0nonbleeding'))
training_data_length = len(files1) + len(files2)

In [ ]:
_, _, files1 = next(os.walk(f'{validation_path}/1bleeding'))
_, _, files2 = next(os.walk(f'{validation_path}/0nonbleeding'))
validation_data_length = len(files1) + len(files2)

In [ ]:
print(f'Training data length: {training_data_length}; validation data length: {validation_data_length}')

In [ ]:
#Or use this

In [ ]:
#training data
training_path_non_bleeding = '/content/drive/My Drive/CNN_LIME-master/training/training/0nonbleeding'
training_path_bleeding = '/content/drive/My Drive/CNN_LIME-master/training/training/1bleeding'

In [ ]:
#validation data
validation_path_non_bleeding = '/content/drive/My Drive/CNN_LIME-master/training/validation/0nonbleeding'
validation_path_bleeding = '/content/drive/My Drive/CNN_LIME-master/training/validation/1bleeding'

## Training
Now, we save the model and save it to disk. For this, we use this Keras code as the blue print: https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d.

In [35]:
# dimensions of our images.
img_width, img_height = 150, 150

nb_train_samples = training_data_length
nb_validation_samples = validation_data_length
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    training_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

hist = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('model.h5')

Found 2993 images belonging to 2 classes.
Found 302 images belonging to 2 classes.
Epoch 1/50
187/187 [==============================] - 76s 405ms/step - loss: 0.3041 - acc: 0.8880 - val_loss: 0.1054 - val_acc: 0.9618
Epoch 2/50
187/187 [==============================] - 75s 400ms/step - loss: 0.1132 - acc: 0.9666 - val_loss: 0.1341 - val_acc: 0.9479
Epoch 3/50
187/187 [==============================] - 75s 399ms/step - loss: 0.1177 - acc: 0.9733 - val_loss: 0.1909 - val_acc: 0.9618
Epoch 4/50
187/187 [==============================] - 75s 399ms/step - loss: 0.0855 - acc: 0.9789 - val_loss: 0.4424 - val_acc: 0.9444
Epoch 5/50
187/187 [==============================] - 76s 405ms/step - loss: 0.0859 - acc: 0.9806 - val_loss: 0.2141 - val_acc: 0.9688
Epoch 6/50
187/187 [==============================] - 74s 395ms/step - loss: 0.1180 - acc: 0.9750 - val_loss: 0.1045 - val_acc: 0.9688
Epoch 7/50
187/187 [==============================] - 74s 395ms/step - loss: 0.0855 - acc: 0.9843 - val_los

In [36]:
model.save('model_full.h5')